# Groupby and Arrest Data

In our merging exercises, we examined the relationship between county-level violent arrest totals and county-level drug arrest totals. In those exercises, you were given a dataset that provided you with county-level arrest totals. But that's not actually how the data is provided by the state of California. This week we will work with the *raw* California arrest data, which is not organized by county or even county-year. 



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_groupby.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex4_num_rows",
    "ex5_collapsed_vars",
    "ex7_alameda_1980_share_violent_arrestees_black",
    "ex11_white_drug_share",
    "ex11_black_drug_share",
    "ex12_proportionate",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

In [5]:
import numpy as np
import pandas as pd

pd.set_option("mode.copy_on_write", True)
results = {}

### Exercise 1

Import the raw California arrest data from the State Attorney General's office. Please use [this link](https://github.com/nickeubank/MIDS_Data/blob/master/OnlineArrestData1980-2021.csv) (the original is here [here](https://openjustice.doj.ca.gov/data), but they keep updating it and I get tired of updating solutions, so... please use my copy!)

In [6]:
df = pd.read_csv(
    "https://media.githubusercontent.com/media/nickeubank/MIDS_Data/master/OnlineArrestData1980-2021.csv"
)
df

,YEAR,GENDER,RACE,AGE_GROUP,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
0,1980,Male,Black,Under 18,Alameda County,505,1351,188,26,79,2149,2286,295
1,1980,Male,Black,18 to 19,Alameda County,205,465,183,8,48,909,1333,0
2,1980,Male,Black,20 to 29,Alameda County,949,1593,606,27,178,3353,7974,0
3,1980,Male,Black,30 to 39,Alameda County,450,755,241,18,110,1574,4876,0
4,1980,Male,Black,40 to 69,Alameda County,172,218,117,11,66,584,3836,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102240,2021,Female,White,70 and over,Tulare County,0,0,1,0,0,1,15,0
102241,2021,Female,White,70 and over,Tuolumne County,2,0,0,0,0,2,6,0
102242,2021,Female,White,70 and over,Ventura County,2,0,0,0,0,2,29,0
102243,2021,Female,White,70 and over,Yolo County,0,0,0,0,0,0,4,0


## Learning the Group Structure of Your Data

### Exercise 2

What is the unit of observation for this dataset? In other words, when row zero says that there were 505 arrests for `VIOLENT` crimes, what exactly is that telling you—505 arrests in 1980? 505 arrests in Alameda County?

(Please answer in Markdown)

In [7]:
df.sample(10)

,YEAR,GENDER,RACE,AGE_GROUP,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
84864,2014,Male,Black,20 to 29,Yolo County,20,17,23,4,31,95,121,0
38639,1996,Female,Hispanic,70 and over,Los Angeles County,1,0,3,0,0,4,33,0
77330,2011,Female,Hispanic,18 to 19,Tulare County,13,24,7,2,15,61,142,0
5808,1982,Female,White,18 to 19,Plumas County,0,0,0,0,1,1,1,0
84549,2014,Female,Black,18 to 19,Sonoma County,1,0,1,1,0,3,6,0
11037,1984,Male,Black,20 to 29,Siskiyou County,2,1,0,0,0,3,9,0
38660,1996,Male,Hispanic,20 to 29,Madera County,172,78,33,5,61,349,1184,0
27541,1991,Male,Hispanic,40 to 69,Shasta County,5,0,3,1,4,13,23,0
90237,2017,Male,White,18 to 19,El Dorado County,8,5,2,0,3,18,49,0
19989,1988,Female,Black,40 to 69,San Diego County,18,30,26,0,3,77,258,0


We believe that the rows show the arrest data for each race's age group, for a specific year, gender, and county in California.

### Exercise 3

Use `duplicated` to test if the variables *you* think uniquely identify rows in your data really do uniquely identify rows. If you were wrong, update your beliefs (and your code) until you have an `assert` statement that passes.

In [8]:
assert not df.duplicated(["YEAR", "GENDER", "RACE", "AGE_GROUP", "COUNTY"]).any()

### Exercise 4

Once you have a handle on how the data looks now, please **collapse the data** to be one observation per county-year-racial group. Store the number of rows in the resulting dataframe in your `results` dictionary under the key `"ex4_num_rows"`.

**Hint:** Think carefully about the most appropriate *aggregation function* (e.g., mean, median, sum, count) given the data we're working with!

**Hint 2:** When using `groupby`, I am a HUGE fan of the `as_index=False` keyword argument (e.g., `df.groupby("col1", as_index=False).mean()`). By default when you use groupby, pandas likes to make a new index out of the grouping columns. As you know, I HATE indices, and this keyword argument prevents pandas from this behavior. To illustrate:

In [11]:
print(
    "The following below shows the total count of violent crimes committed by each race, each year, in each county. This is regardless of gender or age"
)
osama = df.groupby(["COUNTY", "YEAR", "RACE"], as_index=False)[["VIOLENT", "F_DRUGOFF"]].sum()
osama

The following below shows the total count of violent crimes committed by each race, each year, in each county. This is regardless of gender or age


,COUNTY,YEAR,RACE,VIOLENT,F_DRUGOFF
0,Alameda County,1980,Black,2594,1489
1,Alameda County,1980,Hispanic,592,361
2,Alameda County,1980,Other,139,69
3,Alameda County,1980,White,1179,1650
4,Alameda County,1981,Black,2753,1460
...,...,...,...,...,...
9697,Yuba County,2020,White,179,69
9698,Yuba County,2021,Black,45,12
9699,Yuba County,2021,Hispanic,84,20
9700,Yuba County,2021,Other,11,12


In [12]:
results["ex4_num_rows"] = osama.shape[0]
print(f'The number of rows in this dataframe is {results["ex4_num_rows"]}')

The number of rows in this dataframe is 9702


### Exercise 5

You should notice that there are significantly fewer rows in the data now. Given your answer from 3, what groups were you *collapsing* in question 4 (in other words, which different groups were previously split out into separate rows but have now been aggregated into the same rows)?

Store the names of those variables **in a `list` in alphabetical order** in `results` as `"ex5_collapsed_vars"`.

In [13]:
forgotten = ["AGE_GROUP", "GENDER"]
results["ex5_collapsed_vars"] = forgotten

## Racial Arrest Composition

The next question we want to ask is: does the racial composition of arrests in each county vary by arrest type? In other words, do Blacks, Hispanics, Whites, and Other make up substantially different portions of the people arrested for drug offenses and violent offenses?

To answer this question, we need to be able to say what proportion of all drug arrestees were Black/White/Hispanic/Other and what proportion of all violent arrestees were Black/White/Hispanic/Other *for each county and for each year*.

In trying to do this, we'll need to break the problem down into pieces, starting from our goal (these quantities) and working backwards.

### Exercise 6

To calculate the share of all drug and violent arrestees who come from each racial group, what quantities do we need? 

Expressed differently, what two variables (one for drug arrests, one for violent arrests) do we need to add to our data to it allow us to calculate the share of arrestees in each county-year that come from a given racial group?

Calculate these two variables (one variable for drug arrests, one for violent arrests). 

**Hint:** `transform` should probably make an appearance here.

In [15]:
osama["total_d"] = osama.groupby(['COUNTY', 'YEAR'])["F_DRUGOFF"].transform("sum")
osama["total_v"] = osama.groupby(['COUNTY', 'YEAR'])["VIOLENT"].transform("sum")
osama

,COUNTY,YEAR,RACE,VIOLENT,F_DRUGOFF,total_d,total_v
0,Alameda County,1980,Black,2594,1489,3569,4504
1,Alameda County,1980,Hispanic,592,361,3569,4504
2,Alameda County,1980,Other,139,69,3569,4504
3,Alameda County,1980,White,1179,1650,3569,4504
4,Alameda County,1981,Black,2753,1460,3926,4699
...,...,...,...,...,...,...,...
9697,Yuba County,2020,White,179,69,104,311
9698,Yuba County,2021,Black,45,12,112,355
9699,Yuba County,2021,Hispanic,84,20,112,355
9700,Yuba County,2021,Other,11,12,112,355


In [16]:
sum(osama["total_d"] == 0)

17

**Hint:** This types of groupby manipulations are an easy place to do things wrong, so it's important to eye-ball your data to be sure you did things right! 

Take a look at the data for, say, Alameda County in 1980, and compare the values that you calculate above with what you get if you try to calculate those same numbers by hand using the raw data from Exercise 5?

### Exercise 7

Now calculate the share of violent and drug arrestees in each county and year from each racial group.

To ensure you're on the right track, store the share (between zero and one) of violent arrestees in Alameda County in 1980 who were Black as `ex7_alameda_1980_share_violent_arrestees_black`.

In [15]:
df["race_violent"] = df["VIOLENT"] / df["total_v"]
df["race_drug"] = df["F_DRUGOFF"] / df["total_d"]

In [47]:
share_arrests = df.groupby(["YEAR", "COUNTY", "RACE"])[
    ["race_violent", "race_drug"]
].sum()
share_arrests.reset_index(inplace=True)

In [48]:
share_arrests

,YEAR,COUNTY,RACE,race_violent,race_drug
0,1980,Alameda County,Black,0.575933,0.417204
1,1980,Alameda County,Hispanic,0.131439,0.101149
2,1980,Alameda County,Other,0.030861,0.019333
3,1980,Alameda County,White,0.261767,0.462314
4,1980,Alpine County,Other,0.000000,0.000000
...,...,...,...,...,...
9697,2021,Yolo County,White,0.356250,0.554348
9698,2021,Yuba County,Black,0.126761,0.107143
9699,2021,Yuba County,Hispanic,0.236620,0.178571
9700,2021,Yuba County,Other,0.030986,0.107143


In [49]:
results["ex7_alameda_1980_share_violent_arrestees_black"] = share_arrests.loc[0][
    "race_violent"
]
results

{'ex4_num_rows': 9702,
 'ex5_collapsed_vars': ['AGE_GROUP', 'GENDER'],
 'ex7_alameda_1980_share_violent_arrestees_black': 0.5759325044404974}

### Exercise 8

We're about to start studying this data by plotting the share of violent arrestees that are Black against the share of felony drug arrestees that are Black. But the moment where you finish your data manipulations and are about to start you data analysis is a *great* time to just make sure everything in your data looks good. Let's run a few checks:

- Are your values of the share of felony arrestees who were arrested for violent crimes sensible?
- You're about to analyze the data using only the rows for the Black racial group. How many unique counties are there with data for Black arrestees? How many for White arrestees? Do you remember how many counties there are in CA (google is your friend if not!)?

Include assert statements to check all these features of your data.

(Reminder: the autograder will only work if you are entire notebook runs without errors.)

If you encounter any problems, make sure you understand why they are happening, and fix them (if a fix is appropriate!).

In [57]:
black_arrests = share_arrests[share_arrests["RACE"] == "Black"]
white_arrests = share_arrests[share_arrests["RACE"] == "White"]
print("This cell seperates the data into an all black and all white dataframe")

This cell seperates the data into an all black and all white dataframe


In [58]:
assert len(black_arrests["COUNTY"].unique()) == 58
assert len(white_arrests["COUNTY"].unique()) == 58

### Exercise 9

Now check to see if you have the same number of counties *in each year for each racial group*.

Can you figure out why there aren't? (Hint: this isn't trivial to figure out! You'll probably have to do some detective work, and even then you'll have to make a bit of a "best guess" at what's going on!)

In [65]:
ex9 = share_arrests.groupby(["YEAR", "RACE"])["COUNTY"].count().reset_index()
ex9

,YEAR,RACE,COUNTY
0,1980,Black,55
1,1980,Hispanic,57
2,1980,Other,58
3,1980,White,58
4,1981,Black,55
...,...,...,...
163,2020,White,58
164,2021,Black,58
165,2021,Hispanic,58
166,2021,Other,58


In [74]:
investigate_new = share_arrests[(share_arrests["YEAR"] == 1980)]
investigate_new

,YEAR,COUNTY,RACE,race_violent,race_drug
0,1980,Alameda County,Black,0.575933,0.417204
1,1980,Alameda County,Hispanic,0.131439,0.101149
2,1980,Alameda County,Other,0.030861,0.019333
3,1980,Alameda County,White,0.261767,0.462314
4,1980,Alpine County,Other,0.000000,0.000000
...,...,...,...,...,...
223,1980,Yolo County,White,0.634518,0.787037
224,1980,Yuba County,Black,0.097561,0.038462
225,1980,Yuba County,Hispanic,0.103659,0.012821
226,1980,Yuba County,Other,0.018293,0.012821


In [78]:
sum(df["F_DRUGOFF"].isna())

0

### Exercise 10

Once you think you have a theory of the case, fix the problem. Again, this will NOT be trivial.

### Exercise 11

Now that you have corrected your problem, averaging over the values for each county-year, calculate the average share of drug arrestees who are Black and the average share of drug arrestees who are White. Store the results as `"ex11_white_drug_share"` and `"ex11_black_drug_share"`.

### Exercise 12

Plot the share of violent arrestees that are Black against the share of felony drug arrestees that are Black. Do they look proportionate? Answer in text here.

Do Black arrestees make up a greater share of violent arrestees, or a greater share of drug arrestees? Store you answer as `"greater violent"`, or `"greater drug"` in `"ex12_proportionate"`.

**Hint:** You can add a 45 degree reference line by making a dummy dataset with one point at 0,0 and one point at a location near the top of your data (e.g. 0.7, 0.7). Without a reference line it's VERY hard to be sure if how your points relate to proportionality.

(A quick note of warning on interpretation: these results can tell you whether Black Californians make up a larger proportion of *arrests* for certain types of crimes, not whether they make up a larger proportion of people who *commit* a give type of crime! Those *might* be the same, but they might not... this data just can't answer that question.)